In [ ]:
import numpy as np
from collections import defaultdict
import random

# =========================
# 1. ゲーム設定（元コードと同じ）
# =========================
cards = [0,1,2]   # J=0, Q=1, K=2
card_names = ['J','Q','K']
actions = [0,1]   # Check/Fold=0, Bet/Raise/Call=1
action_names = ['Check/Fold','Raise/Call']

class Node:
    def __init__(self):
        self.N = 0
        self.Q = 0
        self.children = {}  # action -> Node

trees1 = defaultdict(Node)
trees2 = defaultdict(Node)

def compute_reward(my_card, opp_card, history):
    reward = 0
    if len(history)==2:
        h0,h1 = history
        if h0==0 and h1==0: #Check→Check
            reward = 1 if my_card>opp_card else -1
        elif h0==1 and h1==0: #Raise→Fold
            reward = 1
        elif h0==1 and h1==1: #Raise→Call
            reward = 2 if my_card>opp_card else -2
    elif len(history)==3: #Check→Raise→?
        h0,h1,h2 = history
        if h0==0 and h1==1:
            if h2==0:#先手アクションがFold
                reward = -1
            else:#先手アクションがCall
                reward = 2 if my_card>opp_card else -2
    return reward

# reward for player_id: keeps same semantics as before
def get_reward_for_player(player_id, p1_card, p2_card, history):
    p1_view = compute_reward(p1_card, p2_card, history)
    return p1_view if player_id == 1 else -p1_view

def is_terminal(history):
  if history in ([0,0],[1,0],[1,1],[0,1,0],[0,1,1]):
    return True
  return False

# =========================
# 新しいシミュレーション関数（両プレイヤーがUCBで行動選択し学習する）
# =========================
def simulate_both_mcts():
    """
    1プレイアウトで両者とも自己のツリーを用いてUCB選択を行い、
    プレイアウト終了後に各プレイヤーの訪問情報でバックアップする。
    """
    # サンプリングされた private cards（1回のプレイアウトにつき固定）
    p1_card = np.random.choice(cards)
    p2_card = np.random.choice([c for c in cards if c != p1_card])

    history = []

    # 訪問記録: player -> list of (node, action, child_node)
    visits_p1 = []
    visits_p2 = []

    # どちらのプレイヤが選択するかは履歴長に依存する
    # 履歴が偶数長ならP1の番（履歴長=0,2 -> P1 が決定。注意: 履歴長=2は通常ゲーム終了だが条件で扱う）
    # 履歴が奇数長ならP2の番
    # ただし、このゲームの終端は履歴長が2または3になるタイミングなのでループでそれをチェック
    Cp = 2.0

    while True:
        if is_terminal(history) == True:
          break

        # 決定者を決める
        current_player = 1 if len(history) % 2 == 0 else 2

        # 状態キーは (player_card, tuple(history))
        if current_player == 1:
            state = (p1_card, tuple(history))
            tree = trees1
            visits = visits_p1
        else:
            state = (p2_card, tuple(history))
            tree = trees2
            visits = visits_p2

        node = tree[state]  # defaultdict によってノードが自動生成される

        # 子が未生成／未訪問の行動がある場合はランダムに選ぶ（探索促進）
        if node.N == 0 or any(a not in node.children or node.children[a].N == 0 for a in actions):
            action = np.random.choice(actions)
        else:
            # UCB 選択
            action = max(actions,
                     key=lambda a: node.children[a].Q + np.sqrt(np.log(node.N + 1) / (node.children[a].N + 1)*Cp))

        # ノードとactionを記録しておき、バックアップ時に使う
        # 子ノードがなければ作る（即座にN/Q更新可能）
        if action not in node.children:
            node.children[action] = Node()
        child_node = node.children[action]

        visits.append((node, action, child_node))

        # 実際に行動を反映
        history = history + [action]

        # 特殊ケース: もし Check -> Raise -> (P1 の返し) などで次がゲーム終了になればループが抜ける

        # ループ続行して次のプレイヤの判断へ（両者とも同じロジックで選択を行う）
        # ここでは"ロールアウト"部を単純化しており、ランダムポリシーや固定ロールアウトは入れていない。
        # （よりPOMCPに忠実にするには、未展開ノードに出会ったら確率的ロールアウトを走らせる実装を追加できます。）

    # ゲーム終了: 報酬を計算（P1視点）
    reward_p1 = compute_reward(p1_card, p2_card, history)

    # 各訪問リストについてバックアップ（各プレイヤー視点の報酬で）
    # P1 の訪問更新（報酬は reward_p1）
    for node, action, child in visits_p1:
        child.N += 1
        # 子の Q を incremental update（平均）で更新
        child.Q += (reward_p1 - child.Q) / child.N
        node.N += 1

    # P2 の訪問更新（P2視点の報酬は -reward_p1）
    for node, action, child in visits_p2:
        child.N += 1
        child.Q += ((-reward_p1) - child.Q) / child.N
        node.N += 1

    # 戻り値は P1 視点の報酬（必要なら呼び出し側で使う）
    return reward_p1

# =========================
# メインループ（両者同時に学習）
# =========================
n_simulations = 100000000  # 必要に応じて増やしてください（100kでも可だが実行時間に注意）
checkpoints = [100, 1000, 10000, 100000, 1000000,2000000,3000000,4000000,5000000,6000000,7000000,8000000,9000000,10000000,100000000]
rewards_all = []

for i in range(n_simulations):
    simulate_both_mcts()
    if (i+1) in checkpoints:
      # =========================
      # 結果表示（元の出力部をそのまま使用できます）
      # 以下は元のコードと同様の出力ロジックを使って trees1 / trees2 を表示できます
      # （出力部分は長いので省略せずに使ってください — ここでは例として先手の分布を表示）
      # =========================

      print("先手の学習済み行動分布n=",i+1)
      for state, node in trees1.items():
          my_card, history = state
          total_N = sum(child.N for child in node.children.values())
          print(f"\nCard={card_names[my_card]}, History={history}")
          for a, child in node.children.items():
              prob = child.N/total_N if total_N>0 else 0
              print(f"  Action {action_names[a]}: N={child.N}, Estimated Prob={prob:.3f}")

      print("後手の学習済み行動分布n=",i+1)
      for state, node in trees2.items():
          my_card, history = state
          if len(history) == 1:
              total_N = sum(child.N for child in node.children.values())
              print(f"\nCard={card_names[my_card]} (P2), History={history}")
              for a, child in node.children.items():
                  prob = child.N/total_N if total_N>0 else 0
                  print(f"  Action {action_names[a]} (P2): N={child.N}, Estimated Prob={prob:.3f}")


先手の学習済み行動分布n= 100

Card=Q, History=()
  Action Check/Fold: N=21, Estimated Prob=0.700
  Action Raise/Call: N=9, Estimated Prob=0.300

Card=Q, History=(np.int64(0), np.int64(1))
  Action Raise/Call: N=8, Estimated Prob=0.889
  Action Check/Fold: N=1, Estimated Prob=0.111

Card=K, History=()
  Action Check/Fold: N=21, Estimated Prob=0.583
  Action Raise/Call: N=15, Estimated Prob=0.417

Card=K, History=(np.int64(0), np.int64(1))
  Action Raise/Call: N=5, Estimated Prob=0.833
  Action Check/Fold: N=1, Estimated Prob=0.167

Card=J, History=()
  Action Check/Fold: N=31, Estimated Prob=0.912
  Action Raise/Call: N=3, Estimated Prob=0.088

Card=J, History=(np.int64(0), np.int64(1))
  Action Raise/Call: N=2, Estimated Prob=0.182
  Action Check/Fold: N=9, Estimated Prob=0.818
後手の学習済み行動分布n= 100

Card=J (P2), History=(np.int64(0),)
  Action Raise/Call (P2): N=9, Estimated Prob=0.300
  Action Check/Fold (P2): N=21, Estimated Prob=0.700

Card=Q (P2), History=(np.int64(0),)
  Action Raise/Call (P2):

KeyboardInterrupt: 